In [14]:
%matplotlib inline
import os
import numpy as np
import pandas as pd
import geopandas as gpd
import matplotlib.pyplot as plt
from rasterio import open as r_open
from rasterio.plot import show as r_show 
from subprocess import Popen
from rasterstats import zonal_stats
import shapely
from shapely.geometry import shape, mapping
from geopandas.tools import sjoin
import requests 
import json

In [15]:
wf = lambda x: '/Users/momol/BU/BUXC433-BayState/data/' + x
rf = lambda x: '/Users/momol/BU/BUXC433-BayState/results/' + x

In [16]:
approved_proj = pd.read_csv(wf('approved_projects.csv'),index_col=False)
census = pd.read_csv(wf('census.csv'),index_col=False)
zba = pd.read_csv(wf('zba.csv'),index_col=False)
MA_PARCELS_df = gpd.read_file(wf('Parcels_2020-shp/Parcels_2020.shp'), encoding='utf-8')

/Users/momol/opt/anaconda3/lib/python3.8/site-packages/IPython/core/interactiveshell.py:3146: DtypeWarning: Columns (17) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


In [30]:
approved_proj["city"].replace({"Boston":"Boston", "West Roxbury":"West Roxbury", "Charlestown":"Charlestown", "Roxbury":"Roxbury", "Brighton":"Brighton",
       "Allston":"Allston", "Jamaica Plain":"Jamaica Plain", "East Boston":"East Boston", "Dorchester":"Dorchester",
       "Hyde Park":"Hyde Park", "South Boston":"South Boston", "Roslindale":"Roslindale", "BOSTON":"Boston",
       "Mission Hill":"Mission Hill", "Mattapan":"Mattapan", "Chestnut Hill":"Chestnut Hill", "NorthEnd":"North End",
       "West End":"West End", "Dorchester/":"Dorchester", "Roxbury/":"Roxbury", "Fenway":"Fenway", "Boston/Roxbury":"Roxbury",
       "ROXBURY":"Roxbury", "South End":"South End", "BOSTON/":"Boston", "Charlestown/":"Charlestown",
       "Back Bay/Boston":"Back Bay", "Boston/":"Boston", "Fenway/":"Fenway", "Boston/Downtown":"Downtown",
       "Boston's Historic North End":"North End", "Dorchester Center/":"Dorchester",
       "Dorchester Center":"Dorchester", "Financial District":"Downtown",
       "Downtown/Financial District":"Downtown", "Dorchester (Lower Mills)/":"Dorchester",
       "BRIGHTON":"Brighton", "ALLSTON":"Allston", "Dorchester//":"Dorchester", "Boston/Beacon Hill":"Beacon Hill",
       "BOSTON/CHINATOWN":"Chinatown", "Boston/Chinatown":"Chinatown", "Chinatown":"Chinatown",
       "SOUTH BOSTON":"South Boston", "Hyde Park/": "Hyde Park", "South End/":"South Boston", "BRIGHTON/":"Brighton",
       "HYDE PARK":"Hyde Park", "East Boston/":"East Boston", "MATTAPAN":"Mattapan", "East  Boston":"East Boston",
       "WEST ROXBURY":"West Roxbury", "West End/":"West End", "Theater District":"Downtown", "Mattapan/":"Mattapan",
       "Brighton/":"Brighton", "CHARLESTOWN":"Charlestown", "Allston/Boston":"Allston", "ROSLINDALE":"Roslindale",
       "West Roxbury/":"West Roxbury", "Boston/Back Bay": "Back Bay", "JAMAICA PLAIN":"Jamaica Plain", "Back Bay/":"Back Bay",
       "dorchester":"Dorchester", "FENWAY":"Fenway", "South Boston/South Boston":"South Boston",
        "DORCHESTER/":"Dorchester", "Jamaica Plain/Boston":"Jamaica Plain",
       "JAMAICA PLAIN/Boston":"Jamaica Plain", "BOSTON/Waterfront":"South Boston Waterfront", "ROXBURY CROSSIN":"Roxbury Crossing",
       "Charlestown/Boston":"Charlestown", "BOSTON/Boston":"Boston", "Boston/South End":"South End",
       "Boston/South":"South End", "Charleswtown":"Charlestown", "Brighton#":"Brighton", "Hyde PArk":"Hyde Park",
       "Financial District/":"Downtown", "Boston/Fenway/Kenmore/Audubo":"Fenway",
       "Roslindale/":"Roslindale", "Allston/":"Allston", "Boston/North End":"North End", "boston":"Boston",
       "Chinatown/":"Chiinatown", "Kenmore/fenway":"Fenway",
       "Boston/Charlestown":"Charlestown", "MATTAPAN ":"Mattapan", "Beacon Hill/":"Beacon Hill", "EAST BOSTON":"East Boston",
       "roxbury":"Roxbury", "Boston/Roslindale":"Roslindale",
       "Charlestown//":"Charlestown", "Dorchester(Lower Mills)":"Dorchester",
       "Dorchester (Savin Hill)/":"Dorchester", "Beacon Hill//":"Beacon Hill",
       "Mission Hill//":"Mission Hill",
       "E BOSTON":"East Boston", "South BostonSG":"South Boston", "Dorchester (Lower Mills)":"Dorchester",
       "Boston/Fenway":"Fenway", "Dorchester Center//":"Dorchester", "CHESTNUT HILL":"Chestnut Hill",
        "Roxbury/Boston":"Roxbury", "south boston":"South Boston", "Brigjton":"Brighton",
       "Ropxbury":"Roxbury", "West Roxbry":"West Roxbury", "Jamaica Plain.":"Jamaica Plain",
       "Brighton//":"Brighton", "West Roxbury/Chestnut Hill":"West Roxbury",
       "Charlestown\\=-0=]\\[=-":"Charlestown", "Jamaica plain":"Jamaica Plain", "East BOSTON":"East Boston",
       "Dorchester (Savin Hill)":"Dorchester", "North End":"North End", "Downtown/Financial":"Downtown",
       "Back /":"Back Bay", "Charlestown+":"Charlestown", "Hyde Park//":"Hyde Park", "BOSTON/SOUTH END":"South End",
       "Charlestown+-+-+-+-+-+":"Charlestown", "South End//":"South End", "North end":"North End",
       "East Boston/Boston/":"Easr Boston", 'NorthEnd/':"North End",
       'DORCHESTER':"Dorchester", 'South Boston/':"South Boston", 'Mission Hill/':"Mission Hill", 'Boston/West End':"West End", 'Chiinatown':"Chinatown",
        "Mission Hill'":"Mission Hill", "Easr Boston":"East Boston", 'Roxbury Crossing':"Mission Hill", 'ROXBURY CROSSING':'Mission Hill', 'NORTH END/WATERFRONT':"North End", 'Brighton/Allston': "Brighton", "Allston / Brighton":"Allston", "Brighton / Allston":"Brighton", "Allston/Brighton":"Allston", ' ':np.nan, '/':np.nan, 'Jamaica Plain/':"Jamaica Plain" }, inplace=True)
approved_proj.city.unique()

array(['Boston', 'West Roxbury', 'Charlestown', 'Roxbury', 'Brighton',
       'Allston', 'Jamaica Plain', 'East Boston', 'Dorchester',
       'Hyde Park', 'South Boston', 'Roslindale', 'Mission Hill',
       'Mattapan', nan, 'Chestnut Hill', 'North End', 'West End',
       'Fenway', 'South End', 'Back Bay', 'Downtown', 'Beacon Hill',
       'Chinatown', 'South Boston Waterfront'], dtype=object)

In [39]:
approved_proj = approved_proj
census = census.rename(columns={'Neighborhood': 'city'})
zba = zba.rename(columns = {'parent_apno':'permitnumber'})

In [41]:
permitletter = []
nums = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '0']
for pn in approved_proj["permitnumber"]:
    letter = ""
    for i in range(3):
        if pn[i] in nums:
            break
        else:
            letter += pn[i]
    permitletter.append(letter)

approved_proj['permitletter'] = permitletter

In [42]:
quartile = []

first = np.percentile(approved_proj.declared_valuation, 25)
second = np.percentile(approved_proj.declared_valuation, 50)
third = np.percentile(approved_proj.declared_valuation, 75)

for val in approved_proj.declared_valuation:
    if val <= first:
        quartile.append(1)
    elif val > first and val <= second:
        quartile.append(2)
    elif val > second and val <= third:
        quartile.append(3)
    else:
        quartile.append(4)

approved_proj["quartile"] = quartile

In [43]:
MA_PARCELS_df= MA_PARCELS_df.to_crs("EPSG:4326")
approved_proj_gdf = gpd.GeoDataFrame(approved_proj, geometry=gpd.points_from_xy(approved_proj.long, approved_proj.lat))

In [44]:
approved_parcels_df = sjoin(approved_proj_gdf, MA_PARCELS_df, how='left', op='within')
approved_parcels_df

<ipython-input-44-00ae17b616cb>:1: UserWarning: CRS mismatch between the CRS of left geometries and the CRS of right geometries.
Use `to_crs()` to reproject one of the input geometries to match the CRS of the other.

Left CRS: None
Right CRS: EPSG:4326

  approved_parcels_df = sjoin(approved_proj_gdf, MA_PARCELS_df, how='left', op='within')


,permitnumber,worktype,permittypedescr,description,comments,applicant,declared_valuation,total_fees,issued_date,expiration_date,...,PARCEL,PID_LONG,WPD,BATCH,Shape__Len,Shape__Are,Shape__L_1,Shape__A_1,Shape__A_2,Shape__L_2
0,A1000569,INTEXT,Amendment to a Long Form,Interior/Exterior Work,This work is to Amend Permit ALT347244. Elimin...,Patrick Sharkey,36500.00,390.0,2021-01-28 11:29:26,2021-07-28 00:00:00,...,3807,0303807000,03-06-285,NaN,None,None,None,None,3279.383301,311.329674
1,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.00,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,12032,2012032000,20-03-015,NaN,None,None,None,None,610720.249146,3125.512528
2,A1001012,OTHER,Amendment to a Long Form,Other,Amend Alt943748 to erect a roof deck as per pl...,Jusimar Oliveria,5000.00,70.0,2020-06-01 14:08:47,,...,2837,0202837000,02-02-140,NaN,None,None,None,None,2053.684814,200.003757
3,A1001201,INTEXT,Amendment to a Long Form,Interior/Exterior Work,Build steel balcony over garden level with sta...,Andreas Hwang,74295.75,803.0,2019-11-13 13:38:56,2020-05-13 00:00:00,...,2558,0402558000,04-04-045,NaN,None,None,None,None,1547.825684,199.142749
4,A100137,EXTREN,Amendment to a Long Form,Renovations - Exterior,Landscaping/stonework - amending permit #2801/...,NaN,15000.00,206.0,2013-01-03 14:13:09,2013-07-03 00:00:00,...,4944,2204944000,22-09-025,NaN,None,None,None,None,8656.629639,434.953850
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
493278,U4999318,VIOL,Use of Premises,Application to Correct a Violation,Application to correct violation V104442.To al...,Ricardo Sousa,0.00,103.0,2013-08-02 13:10:46,2014-02-05 00:00:00,...,517,1000517000,10-02-020,NaN,None,None,None,None,21129.545898,603.533614
493279,U49993435,OSEAT,Use of Premises,Outside Seating,Outdoor patio / amenity space for commercial o...,Christine McMahon,0.00,50.0,2019-09-25 11:36:11,2020-03-25 00:00:00,...,2717,0302717000,03-06-145,NaN,None,None,None,None,24716.314941,720.981286
493280,U49993435,OSEAT,Use of Premises,Outside Seating,Outdoor patio / amenity space for commercial o...,Christine McMahon,0.00,50.0,2019-09-25 11:36:11,2020-03-25 00:00:00,...,2717,0302717000,03-06-145,NaN,None,None,None,None,24716.314941,720.981286
493281,U49993492,TEMTRL,Use of Premises,Temporary Trailers,Temporary Construction Trailer at Herb Chamber...,Regina Olivieri,1000.00,53.0,2019-09-12 13:07:00,2020-03-12 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [45]:
approved_parcels_census_merge = pd.merge(approved_parcels_df, census, on='city', how='left')
approved_parcels_census_merge

,permitnumber,worktype,permittypedescr,description,comments,applicant,declared_valuation,total_fees,issued_date,expiration_date,...,Female Proportion,lat_y,long_y,Primary Sub-Neighborhood,Secondary Sub-Neighborhood,Teritriary Sub-Neighborhood(s),T-Stop/Commuter Rail,Primary Sub-Neighborhood Cleaned,Secondary Sub-Neighborhood Cleaned,Teritiary Sub-Neighborhood(s) Cleaned
0,A1000569,INTEXT,Amendment to a Long Form,Interior/Exterior Work,This work is to Amend Permit ALT347244. Elimin...,Patrick Sharkey,36500.0,390.0,2021-01-28 11:29:26,2021-07-28 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,0.564310,42.298330,-71.139707,Centre South,South Brookline,NaN,NaN,Centre South,South Brookline,NaN
2,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,0.515819,42.290425,-71.171498,Brook Farm,Oak Hill Park,NaN,NaN,Brook Farm,Oak Hill Park,NaN
3,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,0.523306,42.287332,-71.157012,Brook Farm,NaN,NaN,Highland,Brook Farm,NaN,NaN
4,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,0.536623,42.279129,-71.150710,Bellevue Hill,Roxbury,NaN,Woodard Rd @ Wren,Bellevue Hill,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4885381,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,0.480653,42.320659,-71.104202,Mission Hill,NaN,NaN,Jackson Square/Stony Brook,NaN,NaN,NaN
4885382,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,0.459338,42.319110,-71.108841,Egleston Square,NaN,NaN,Stony Brook,Egleston Square,NaN,NaN
4885383,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,0.546312,42.324256,-71.111211,High Street Hill,Mission Hill,NaN,Heath Street,High Street Hill,NaN,NaN
4885384,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,0.000000,42.298883,-71.126706,Forest Hills/Woodbourne,Stony Brook,NaN,Centre St @ Faulkner Hospital,Forest Hills/Woodbourne,Stony Brook,NaN


In [46]:
approved_parcels_census_merge.to_csv(rf("approved_MAparcels_census.csv"), encoding = "utf-8")

In [47]:
approved_parcels_census_zba_merge = pd.merge(approved_parcels_census_merge, zba, on=['permitnumber'], how='left')
approved_parcels_census_zba_merge

,permitnumber,worktype,permittypedescr,description,comments,applicant,declared_valuation,total_fees,issued_date,expiration_date,...,ever_deferred,num_deferrals,final_decision_date,decision,closed_date,city_y,zip_y,ward,zoning_district,project_description
0,A1000569,INTEXT,Amendment to a Long Form,Interior/Exterior Work,This work is to Amend Permit ALT347244. Elimin...,Patrick Sharkey,36500.0,390.0,2021-01-28 11:29:26,2021-07-28 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,A100071,COB,Amendment to a Long Form,City of Boston,Change connector link layout from attached enc...,Renee Santeusanio,40000.0,429.0,2011-11-04 11:04:58,2012-05-04 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4887815,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4887816,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4887817,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4887818,U4999352,DRIVE,Use of Premises,Driveway Installation,Owners would like to install a curb cut and a ...,Maria Dubrowski,2700.0,53.0,2012-05-30 14:28:59,2012-11-30 00:00:00,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
approved_parcels_census_zba_merge.to_csv(rf("approved_MAparcels_census_zba.csv"), encoding = "utf-8")